Working in colab, gdal needs to be reinstalled with each new connection

- runtime ~30 seconds

In [ ]:
!apt install gdal-bin python3-gdal --quiet

Load in the many different files of the raster that are split into pieces.The output of this is a vrt in order to save space.

In [ ]:

from osgeo import gdal
import glob
import subprocess
gdal.UseExceptions()

# list all files in directory that match pattern
demList = glob.glob("/content/drive/MyDrive/1985_LS_Dec5Try/*.tif")
#demList.insert(0,'/content/drive/MyDrive/2023_AllSagebrush_Landsat/BLANK.tif')
print(demList)
first = gdal.Open(demList[0])
img_reference = first.GetGeoTransform()
x_res = img_reference[1]
y_res = -img_reference[5]

#merge
#cmd = "gdal_merge.py -o mergedDEM.tif"
#subprocess.call(cmd.split()+demList)
options_list = [
    '-hidenodata'
]
options_string = " ".join(options_list)

# build virtual raster and convert to geotiff
vrt = gdal.BuildVRT("MERGE.vrt", demList,options='-hidenodata -srcnodata "0"')
#gdal.Translate("/content/drive/MyDrive/2023_AllSagebrush_Landsat/updatedMerged3.tif",vrt, xRes = x_res, yRes = y_res)
#!gdal_translate -of GTiff "$vrt" /content/drive/MyDrive/2023_AllSagebrush_Landsat/updatedMerged.tif --config CHECK_FREE_SPACE NO
vrt = None

['/content/drive/MyDrive/1985_LS_Dec5Try/1985_ALL_sevenbands-0000000000-0000000000.tif', '/content/drive/MyDrive/1985_LS_Dec5Try/1985_ALL_sevenbands-0000000000-0000009472.tif', '/content/drive/MyDrive/1985_LS_Dec5Try/1985_ALL_sevenbands-0000000000-0000018944.tif', '/content/drive/MyDrive/1985_LS_Dec5Try/1985_ALL_sevenbands-0000000000-0000028416.tif', '/content/drive/MyDrive/1985_LS_Dec5Try/1985_ALL_sevenbands-0000000000-0000037888.tif', '/content/drive/MyDrive/1985_LS_Dec5Try/1985_ALL_sevenbands-0000000000-0000047360.tif', '/content/drive/MyDrive/1985_LS_Dec5Try/1985_ALL_sevenbands-0000000000-0000056832.tif', '/content/drive/MyDrive/1985_LS_Dec5Try/1985_ALL_sevenbands-0000000000-0000066304.tif', '/content/drive/MyDrive/1985_LS_Dec5Try/1985_ALL_sevenbands-0000000000-0000075776.tif', '/content/drive/MyDrive/1985_LS_Dec5Try/1985_ALL_sevenbands-0000009472-0000000000.tif', '/content/drive/MyDrive/1985_LS_Dec5Try/1985_ALL_sevenbands-0000009472-0000009472.tif', '/content/drive/MyDrive/1985_LS

The code below allows for splitting a large raster in numpy arrays where each array is a band. (file extension .npy)

In [ ]:
##OUTDATED
import numpy as np
from osgeo import gdal
in_file = r"/content/MERGE.vrt"
ds = gdal.Open(in_file)
x = ds.GetRasterBand(6).ReadAsArray()
np.save('/content/drive/MyDrive/Drone_Flights/RF_intermediate/2007/landsat2007B6.npy',x)

This code below allows for splitting a **VERY** large (>50 GB) raster band into pieces then saving the pieces as numpy arrays (.npy)

In [ ]:
import numpy as np
from osgeo import gdal
in_file = r"/content/MERGE.vrt"
ds = gdal.Open(in_file)
## Change!
x = ds.GetRasterBand(6).ReadAsArray()
new_shape = (x.shape[0] * x.shape[1],1)
temp_array = x.reshape(new_shape)
split = np.array_split(temp_array,124,axis=0)
from tqdm import tqdm
for i in tqdm(range(0,len(split))):
  # CHANGE!
  path = '/content/drive/MyDrive/RF_prep/1985/b6_splits/b6_split' + str(i) + '.npy'
  np.save(path,split[i])

100%|██████████| 124/124 [06:52<00:00,  3.32s/it]
